In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --------------------------------------------------------
# SETTINGS
# --------------------------------------------------------
RUN_NAME = "cow_run22"   # <--- change this to cow_runV23 later
BASE_DIR = f"../projects/simple_run/results/{RUN_NAME}"

# Where ascending aorta pressure waveform is stored
AORTA_FILE = os.path.join(BASE_DIR, "heart_kim_lit", "p_LV1.txt")

# Where ascending aorta flow is stored (modify if needed)
AORTA_FLOW_FILE = os.path.join(BASE_DIR, "heart_kim_lit", "g1.txt")

# --------------------------------------------------------
# Helper function to load firstblood txt timeseries
# --------------------------------------------------------
def load_timeseries(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Missing file: {path}")

    # Read raw lines
    with open(path, "r") as f:
        raw = f.readlines()

    cleaned = []
    for line in raw:
        # Remove commas, semicolons, tabs
        line = line.replace(",", " ").replace(";", " ").strip()
        if not line:
            continue
        parts = line.split()

        # Must have at least 2 numeric columns
        if len(parts) < 2:
            continue

        try:
            t = float(parts[0])
            v = float(parts[1])
            cleaned.append([t, v])
        except ValueError:
            # Skip lines that still cannot be parsed
            continue

    if len(cleaned) == 0:
        raise ValueError(f"No valid numeric rows found in file: {path}")

    data = np.array(cleaned)
    time = data[:, 0]
    val  = data[:, 1]

    return time, val



# --------------------------------------------------------
# 1. LOAD PRESSURE & FLOW
# --------------------------------------------------------
t_p, p_ao = load_timeseries(AORTA_FILE)
t_q, q_ao = load_timeseries(AORTA_FLOW_FILE)

# convert pressure to mmHg if needed (FirstBlood uses Pa)
p_ao_mmHg = p_ao / 133.322

# --------------------------------------------------------
# 2. BASIC STATISTICS
# --------------------------------------------------------
P_sys = np.max(p_ao_mmHg)
P_dia = np.min(p_ao_mmHg)
P_mean = np.mean(p_ao_mmHg)

# Heart rate
# Find peaks in pressure to detect heartbeats
from scipy.signal import find_peaks

peaks, _ = find_peaks(p_ao_mmHg, distance=50)
if len(peaks) > 1:
    beat_intervals = np.diff(t_p[peaks])
    HR = 60.0 / np.mean(beat_intervals)
else:
    HR = np.nan

# Cardiac Output (integrate one cardiac cycle)
# Convert flow to L/min ; FirstBlood flow is m^3/s
q_Lmin = q_ao * 60 * 1000  # m^3/s -> L/min

# CO by averaging flow over whole simulation
CO = np.mean(q_Lmin)

# Pulse Pressure
PP = P_sys - P_dia


# --------------------------------------------------------
# 3. PRINT RESULTS
# --------------------------------------------------------
print("======= FIRSTBLOOD LEVEL-1 SANITY CHECK =======")
print(f"Simulation Run: {RUN_NAME}\n")

print(f"Peak Systolic Pressure (mmHg): {P_sys:.2f}")
print(f"End Diastolic Pressure (mmHg): {P_dia:.2f}")
print(f"Mean Aortic Pressure (mmHg):   {P_mean:.2f}")
print(f"Pulse Pressure (mmHg):         {PP:.2f}")

print(f"Heart Rate (bpm):              {HR:.2f}")
print(f"Cardiac Output (L/min):        {CO:.2f}")

# Validation thresholds from physiology:
print("\n--- Physiological Ranges (for comparison) ---")
print("Systolic: 110–130 mmHg")
print("Diastolic: 70–90 mmHg")
print("MAP:       85–100 mmHg")
print("CO:        4.5–5.5 L/min")
print("HR:        55–90 bpm")
print("Pulse Pressure: 35–55 mmHg")

# --------------------------------------------------------
# 4. PLOT PRESSURE & FLOW
# --------------------------------------------------------
plt.figure(figsize=(12,5))
plt.plot(t_p, p_ao_mmHg)
plt.title("Aortic Root Pressure (mmHg)")
plt.xlabel("Time (s)")
plt.ylabel("Pressure (mmHg)")
plt.grid()
plt.show()

plt.figure(figsize=(12,5))
plt.plot(t_q, q_Lmin)
plt.title("Aortic Flow (L/min)")
plt.xlabel("Time (s)")
plt.ylabel("Flow (L/min)")
plt.grid()
plt.show()


ValueError: could not convert string '0.0000000e+00,' to float64 at row 0, column 1.